In [ ]:
# ==============================================================================
# PASO 1: IMPORTAR LIBRERÍAS
# ==============================================================================
import os
import glob
import numpy as np
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

print("Librerías importadas correctamente.")

data_folder_path = r"C:\Users\Usuario\Documents\UD\NASa\5" # 

# ------------------------------------------------------------------------------

# Buscamos los archivos .tiff de las polarizaciones VV y VH
try:
    vh_path = glob.glob(os.path.join(data_folder_path, '*vh*.tiff'))[0]
    vv_path = glob.glob(os.path.join(data_folder_path, '*vv*.tiff'))[0]
    print(f"Archivo VH encontrado: {os.path.basename(vh_path)}")
    print(f"Archivo VV encontrado: {os.path.basename(vv_path)}")
except IndexError:
    print(f"ERROR: No se encontraron los archivos .tiff en la carpeta: {data_folder_path}")
    print("Asegúrate de que la ruta sea correcta y que los archivos estén ahí.")


def load_sar_image(path):
    with rasterio.open(path) as src:
        return src.read(1).astype('float32')

# Cargamos los datos
if 'vv_path' in locals() and 'vh_path' in locals():
    vv_band = load_sar_image(vv_path)
    vh_band = load_sar_image(vh_path)
    vv_band[vv_band == 0] = np.nan
    vh_band[vh_band == 0] = np.nan
    print(f"\nImágenes cargadas exitosamente. Dimensiones: {vv_band.shape}")

# ==============================================================================
# PASO 3: ANÁLISIS Y VISUALIZACIÓN
# ==============================================================================
plt.rcParams['figure.figsize'] = (20, 15)

# --- ANÁLISIS 1: COMPOSICIÓN RGB (R:VV, G:VH, B:VV-VH) ---
vv_db = 10 * np.log10(vv_band)
vh_db = 10 * np.log10(vh_band)

def normalize(band):
    return (band - np.nanmin(band)) / (np.nanmax(band) - np.nanmin(band))

R = normalize(vv_db)
G = normalize(vh_db)
B = normalize(vv_db - vh_db)
rgb_composite = np.dstack((R, G, B))

plt.subplot(2, 2, 1)
plt.imshow(rgb_composite)
plt.title("Composición RGB (R:VV, G:VH, B:VV-VH)")
plt.xlabel("Píxeles (X)")
plt.ylabel("Píxeles (Y)")
plt.grid(False)

# --- ANÁLISIS 2: DETECCIÓN DE AGUA ---
water_threshold = -20
water_mask = vv_db < water_threshold

plt.subplot(2, 2, 2)
plt.imshow(water_mask, cmap='Blues')
plt.title(f"Máscara de Agua (Umbral VV < {water_threshold} dB)")
plt.xlabel("Píxeles (X)")
plt.ylabel("Píxeles (Y)")
plt.grid(False)

# --- ANÁLISIS 3: ÍNDICE DE HUMEDAD / VEGETACIÓN (Ratio VH/VV) ---
ratio_vh_vv = vh_band / vv_band

plt.subplot(2, 2, 3)
im = plt.imshow(ratio_vh_vv, cmap='viridis', vmin=np.nanpercentile(ratio_vh_vv, 5), vmax=np.nanpercentile(ratio_vh_vv, 95))
plt.title("Índice de Humedad/Vegetación (Ratio VH/VV)")
plt.xlabel("Píxeles (X)")
plt.ylabel("Píxeles (Y)")
plt.colorbar(im, fraction=0.046, pad=0.04).set_label('Ratio VH/VV')
plt.grid(False)

# --- IMAGEN DE REFERENCIA: VV en dB ---
plt.subplot(2, 2, 4)
show(vv_db, cmap='gray', title='Imagen VV (en dB)')
plt.xlabel("Píxeles (X)")
plt.ylabel("Píxeles (Y)")
plt.grid(False)

plt.tight_layout()
plt.show()

Librerías importadas correctamente.
Archivo VH encontrado: s1a-iw-grd-vh-20250720t104319-20250720t104344-060166-0779eb-002.tiff
Archivo VV encontrado: s1a-iw-grd-vv-20250720t104319-20250720t104344-060166-0779eb-001.tiff

Imágenes cargadas exitosamente. Dimensiones: (16810, 25384)
